In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import random

In [2]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [3]:
# build training data

sift_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'

train_data = []
train_labels = []

n_sample = 1000

for name in labels[1:]:
    train_hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
    train_hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
    train_hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})
    
    n_train = train_hists0.shape[0]

    #     train_hists = np.c_[train_hists0, train_hists1.reshape((n_train, -1)), train_hists2.reshape((n_train, -1))]

    random_indices = np.random.choice(range(n_train), min(n_train, n_sample), replace=False)
    n_train = len(random_indices)
    train_hists = np.c_[train_hists0[random_indices], 
                        train_hists1[random_indices].reshape((n_train, -1)), 
                        train_hists2[random_indices].reshape((n_train, -1))]
    
    train_data.append(train_hists)
    train_labels.append(np.ones((n_train, )) * labels_index[name])

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)
n_train = train_data.shape[0]
    
print train_data.shape

(12464, 4200)


In [4]:
train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':


In [5]:
# build testing data

stack = 'MD585'

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

sec = first_detect_sec

test_hists0 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
test_hists1 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
test_hists2 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})

n_test = test_hists0.shape[0]
test_hists = np.c_[test_hists0, test_hists1.reshape((n_test, -1)), test_hists2.reshape((n_test, -1))]

test_data = test_hists

In [6]:
test_data.shape

(20979, 4200)

In [7]:
test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)

In [24]:
# method 1
t = time.time()

def compute_intersection_kernel_oneJob(i, j1):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                     for i in range(n_train)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 200s / 12k training data

train_dist_mat = np.empty((n_train, n_train))
train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
r = np.tril_indices(n_train)
train_dist_mat[r] = train_dist_mat.T[r]

compute intersection kernel: 192.517379 seconds


In [ ]:
# method 2
t = time.time()

def compute_intersection_kernel_oneJob(h):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized, h).sum(axis=1)
    return dist

# train_dist_mat = np.array(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(h) 
#                                      for h in train_data_normalized[:1000]))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

In [16]:
# method 3
t = time.time()

def compute_intersection_kernel_oneJob(i, ni, j, nj):
    dists = np.minimum(train_data_normalized[i:i+ni, None], train_data_normalized[j:j+nj]).sum(axis=-1)
    return dists

train_dist_mat = np.empty((n_train, n_train))
ni = 100
nj = 100
for j in range(0, n_train, nj):
    train_dist_mat[:, j:j+nj] = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, ni, j, nj) 
                                                    for i in range(0, n_train, ni)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400


compute intersection kernel: 369.700769 seconds


In [19]:
bp.pack_ndarray_file(train_dist_mat, sift_dir + '/train/train_spm_dist_mat.bp')

In [ ]:
class Hypersphere_abstain:
    
    def __init__(self, histogram, threshold, label, alpha, size, acc, index, 
                 unweighted_acc, label_distribution, neighbor_indices):
        
        self.h = histogram
        self.t = threshold
        self.l = label
        self.alpha = alpha
        self.size = size
        self.acc = acc # Weighted accuracy
        self.index = index
        self.unweighted_acc = unweighted_acc
        self.label_distribution = label_distribution
        self.neighbor_indices = neighbor_indices        

In [ ]:
# Adaboost.M2

n_train = train_data.shape[0]

D = 1./ n_train * np.ones((n_train, ))

W = np.ones((n_train, ))

# choose exemplar

# compute accuracy of exemplar

# choose the plurality

# compute threshold

# reweight samples

# store hypersphere


